# Joining on Non-Equi Operators

In [1]:
import pandas as pd
import janitor
import numpy as np

In [2]:
#https://stackoverflow.com/q/61948103/7175713 
df1 = pd.DataFrame({'id': [1,1,1,2,2,3], 
                    'value_1': [2,5,7,1,3,4]})

df2 = pd.DataFrame({'id': [1,1,1,1,2,2,2,3], 
                    'value_2A': [0,3,7,12,0,2,3,1], 
                    'value_2B': [1,5,9,15,1,4,6,3]})


In [3]:
df1

,id,value_1
0,1,2
1,1,5
2,1,7
3,2,1
4,2,3
5,3,4


In [4]:
df2

,id,value_2A,value_2B
0,1,0,1
1,1,3,5
2,1,7,9
3,1,12,15
4,2,0,1
5,2,2,4
6,2,3,6
7,3,1,3


Join on equi and non-equi operators is possible:

In [5]:
df1.conditional_join(
        df2,
        ('id', 'id', '=='),
        ('value_1', 'value_2A', '>='),
        ('value_1', 'value_2B', '<='),
        sort_by_appearance = True
    )

,id_x,value_1,id_y,value_2A,value_2B
0,1,5,1,3,5
1,1,7,1,7,9
2,2,1,2,0,1
3,2,3,2,2,4
4,2,3,2,3,6


The default join is `inner`. left and right joins are supported as well:

In [6]:
df1.conditional_join(
        df2,
        ('id', 'id', '=='),
        ('value_1', 'value_2A', '>='),
        ('value_1', 'value_2B', '<='),
        how='left',
        sort_by_appearance = True
    )


,id_x,value_1,id_y,value_2A,value_2B
0,1,2,NaN,NaN,NaN
1,1,5,1.0,3.0,5.0
2,1,7,1.0,7.0,9.0
3,2,1,2.0,0.0,1.0
4,2,3,2.0,2.0,4.0
5,2,3,2.0,3.0,6.0
6,3,4,NaN,NaN,NaN


In [7]:
df1.conditional_join(
        df2,
        ('id', 'id', '=='),
        ('value_1', 'value_2A', '>='),
        ('value_1', 'value_2B', '<='),
        how='right',
        sort_by_appearance = True
    )

,id_x,value_1,id_y,value_2A,value_2B
0,NaN,NaN,1,0,1
1,1.0,5.0,1,3,5
2,1.0,7.0,1,7,9
3,NaN,NaN,1,12,15
4,2.0,1.0,2,0,1
5,2.0,3.0,2,2,4
6,2.0,3.0,2,3,6
7,NaN,NaN,3,1,3


Join on just the non-equi joins is also possible:

In [8]:
df1.conditional_join(
        df2,
        ('value_1', 'value_2A', '>'),
        ('value_1', 'value_2B', '<'),
        how='inner',
        sort_by_appearance = True
    )

,id_x,value_1,id_y,value_2A,value_2B
0,1,2,3,1,3
1,1,5,2,3,6
2,2,3,2,2,4
3,3,4,1,3,5
4,3,4,2,3,6


The default for the suffixes parameter is `(_x, _y)`, One of the suffixes can be set as `None`; this avoids a suffix on the columns from the relevant dataframe:

In [9]:
df1.conditional_join(
        df2,
        ('value_1', 'value_2A', '>'),
        ('value_1', 'value_2B', '<'),
        how='left',
        sort_by_appearance = True,
        suffixes = (None, '_y')
    )

,id,value_1,id_y,value_2A,value_2B
0,1,2,3.0,1.0,3.0
1,1,5,2.0,3.0,6.0
2,1,7,NaN,NaN,NaN
3,2,1,NaN,NaN,NaN
4,2,3,2.0,2.0,4.0
5,3,4,1.0,3.0,5.0
6,3,4,2.0,3.0,6.0


Join on just equality is also possible, but should be avoided, as Pandas' `merge/join` is more efficient (it uses Pandas' internal merge functions anyway); 

In [10]:
df1.conditional_join(
    df2,
    ('id', 'id', "==")
)

,id_x,value_1,id_y,value_2A,value_2B
0,1,2,1,0,1
1,1,2,1,3,5
2,1,2,1,7,9
3,1,2,1,12,15
4,1,5,1,0,1
5,1,5,1,3,5
6,1,5,1,7,9
7,1,5,1,12,15
8,1,7,1,0,1
9,1,7,1,3,5


Join on not equal -> `!=`

In [11]:
df1.conditional_join(
    df2,
    ('id', 'id', "!=")
).select_columns('id*')

,id_x,id_y
0,1,2
1,1,2
2,1,2
3,1,3
4,1,2
5,1,2
6,1,2
7,1,3
8,1,2
9,1,2


Note that if nulls are present in any of the join columns, the nulls are not returned (similar to how SQL operates):

In [12]:
df1.loc[3, 'id'] = np.nan
df2.loc[5, 'id'] = np.nan

In [13]:
df1

,id,value_1
0,1.0,2
1,1.0,5
2,1.0,7
3,NaN,1
4,2.0,3
5,3.0,4


In [14]:
df2

,id,value_2A,value_2B
0,1.0,0,1
1,1.0,3,5
2,1.0,7,9
3,1.0,12,15
4,2.0,0,1
5,NaN,2,4
6,2.0,3,6
7,3.0,1,3


In [15]:
df1.conditional_join(
    df2,
    ('id', 'id', "!=")
).select_columns('id*')

,id_x,id_y
0,1.0,2.0
1,1.0,2.0
2,1.0,3.0
3,1.0,2.0
4,1.0,2.0
5,1.0,3.0
6,1.0,2.0
7,1.0,2.0
8,1.0,3.0
9,2.0,3.0


Note that if the order is not important, sort_by_appearance can be set to `False` (this is the default)